In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 7.5MB/s 
     |████████████████████████████████| 3.3MB 34.0MB/s 
     |████████████████████████████████| 122kB 54.0MB/s 
     |████████████████████████████████| 1.2MB 41.3MB/s 
     |████████████████████████████████| 8.2MB 47.6MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 225kB 54.0MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 1.8MB 38.4MB/s 
     |████████████████████████████████| 2.1MB 46.5MB/s 
     |████████████████████████████████| 4.2MB 43.5MB/s 
     |████████████████████████████████| 163kB 52.2MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
     |████████████████████████████████| 112kB 60.7MB/s 
     |████████████████████████████████| 112kB 56.9MB/s 
     |████████████████████████████████| 245kB 55.0MB/s 
     |████████████████████████████████| 133kB 53.1MB/s 
     |████████████████████████████████| 102kB 15.0MB/

In [3]:
cd drive/MyDrive/Information\ Retrieval

/content/drive/MyDrive/Information Retrieval


In [ ]:
# tqdm still works, but why test can't show tqdm style output
from tqdm import tqdm
from time import sleep
for i in tqdm(range(10000)):
    sleep(0.01)

100%|██████████| 9999/9999 [01:42<00:00, 97.68it/s]


In [5]:
!python test.py

2021-05-10 20:59:43.938271: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
剩余内存空间memory为 (GB):0.46
-------------------------------------
sample abstract and title shown below
Title: Calculation of prompt diphoton production cross sections at Tevatron and LHC energies

Abstract: A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for 

In [ ]:
!python train.py

2021-05-10 05:00:30.019158: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
剩余内存空间memory为 (GB):0.46
-------------------------------------
sample abstract and title shown below
Title: Calculation of prompt diphoton production cross sections at Tevatron and LHC energies

Abstract: A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for 

In [4]:
!python model.py

Streaming output truncated to the last 5000 lines.
Epochs 7/10. Running Loss:    1.4223:  52% 525/1006 [02:41<02:27,  3.25it/s]
Epochs 7/10. Running Loss:    1.4223:  52% 526/1006 [02:41<02:26,  3.27it/s]
Epochs 7/10. Running Loss:    1.3368:  52% 526/1006 [02:41<02:26,  3.27it/s]
Epochs 7/10. Running Loss:    1.3368:  52% 527/1006 [02:41<02:26,  3.27it/s]
Epochs 7/10. Running Loss:    1.6023:  52% 527/1006 [02:41<02:26,  3.27it/s]
Epochs 7/10. Running Loss:    1.6023:  52% 528/1006 [02:41<02:26,  3.26it/s]
Epochs 7/10. Running Loss:    1.4131:  52% 528/1006 [02:41<02:26,  3.26it/s]
Epochs 7/10. Running Loss:    1.4131:  53% 529/1006 [02:42<02:25,  3.27it/s]
Epochs 7/10. Running Loss:    1.4622:  53% 529/1006 [02:42<02:25,  3.27it/s]
Epochs 7/10. Running Loss:    1.4622:  53% 530/1006 [02:42<02:25,  3.28it/s]
Epochs 7/10. Running Loss:    1.3967:  53% 530/1006 [02:42<02:25,  3.28it/s]
Epochs 7/10. Running Loss:    1.3967:  53% 531/1006 [02:42<02:24,  3.28it/s]
Epochs 7/10. Running Loss

In [ ]:
import logging

from tqdm import tqdm
import random
import pandas as pd
from simpletransformers.t5 import T5Model
from dataprocess import getpaper

# --------------------------引入模型部分-----------------------#

'''
这里使用simpletransformers库训练T5模型,即：Text-to-Text Transfer Transformer
'''
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

papers = getpaper()

eval_df = papers.sample(frac=0.2, random_state=101)  # 划分测试集
train_df = papers.drop(eval_df.index)  # 划分训练集

print(train_df.shape, eval_df.shape)  # 打印训练集，测试集
train_df['prefix'] = "summarize"
eval_df['prefix'] = "summarize"

'''
模型参数
'''

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 4,
    "num_train_epochs": 4,
}

# -----------------------Create T5 Model-------------------#


model = T5Model(model_name="t5-small", model_type='t5', args=model_args, use_cuda=True)

# --------------------------测试部分---------------------------#
print('---------------------------------------')
print('testing started')
eval_len = len(eval_df)
seed = random.randint(1,1000)
print(f'random seed is {seed}')
random.seed = seed
actual_titles = []
actual_abstracts = []
predict_titles = []
predict_title = model.predict(["summarize: An example of a short story using ab ovo technique might be Hansel and Gretel, as we really start in the beginning in the action. An example of a short story using in medias res might be The Lottery by Shirley Jackson, as we're already in the middle of the action as the story begins."])
print(predict_title)
# for i in tqdm(range(1, 101), desc='collecting test inputs'):
#     random_num = random.randint(1, eval_len)
#     actual_titles.append(eval_df.iloc[random_num]['target_text'])
#     actual_abstracts.append("summarize: " + eval_df.iloc[random_num]['input_text'])

# predict_titles = model.predict(actual_abstracts)

# test_df = pd.DataFrame({
#     'predict_title': predict_titles,
#     'actual_title': actual_titles,
#     'actual_abstract': actual_abstracts
# })

# test_result = test_df.to_json('./test_result.json', orient="index", index=False)


# for i in range(1, 101):
#     random_num = random.randint(1,eval_len)  # 选定某一个句子（一个数字代表一个sample）
#     actual_title = eval_df.iloc[random_num]['target_text']
#     actual_abstract = ["summarize: " + eval_df.iloc[random_num]['input_text']]
#     predicted_title = model.predict(actual_abstract)[0].replace('.', '')
    
#     # ------------------------输出-------------------------#
#     print('---------------------------------------')
#     print(f'{i}th test sample uses eval data No.{random_num}')
#     print(f'Actual Title: {actual_title}')
#     print(f'Predicted Title: {predicted_title}')
#     print(f'Actual Abstract: {actual_abstract[0]}')

剩余内存空间memory为 (GB):0.51
-------------------------------------
sample abstract and title shown below
Title: Calculation of prompt diphoton production cross sections at Tevatron and LHC energies

Abstract: A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for distributions of diphoton pairs produced at the energy of the Large Hadron Collider . Distributions of the diphoton pairs from the decay of a Higg

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Function [getpaper] run time is 49.88s
(14298, 2) (3574, 2)


INFO:filelock:Lock 139903400093968 acquired on /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985.lock


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

...

---------------------------------------
testing started
random seed is 319


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['an example of a short story using ab ovo technique might be Hansel and']
